<a href="https://colab.research.google.com/github/abxda/UP_Python_2025/blob/main/Semana_04_02_Miercoles_UP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Carga el archivo inegi_analisis_01.duckdb

In [ ]:
# Celda 1: Instalaciones
# ------------------------------------------------------------
!pip install pandas geopandas duckdb requests tqdm shapely --quiet

print("Librerías base instaladas.")

In [ ]:
# Celda 2: Importaciones
# ----------------------
import os
import shutil
import time
from zipfile import ZipFile
import requests
from tqdm import tqdm
import duckdb
import geopandas as gpd
import pandas as pd
from shapely import wkb, wkt # Para manejar geometrías
from shapely.geometry import Point # Para crear puntos a partir de lat/lon

print("Librerías importadas.")

In [ ]:
# Celda 3: Función de Descarga (reutilizada)
# ------------------------------------------
def download(url, directory, filename_override=None):
    """
    Descarga un archivo desde la URL especificada y lo guarda en 'directory'.
    Si el archivo ya existe, no realiza la descarga de nuevo.
    """
    os.makedirs(directory, exist_ok=True)
    filename = filename_override if filename_override else url.split('/')[-1]
    filepath = os.path.join(directory, filename)

    if os.path.exists(filepath):
        print(f"El archivo '{filename}' ya existe en '{directory}'. No se descarga de nuevo.")
        return filepath

    print(f"Descargando '{filename}' de '{url}'...")
    try:
        response = requests.get(url, stream=True, timeout=60)
        response.raise_for_status()
        total_size_in_bytes = int(response.headers.get('content-length', 0))

        progress_bar_params = {
            'desc': filename,
            'total': total_size_in_bytes,
            'unit': 'B', 'unit_scale': True, 'unit_divisor': 1024, 'ncols': 80
        }
        if total_size_in_bytes == 0: progress_bar_params.pop('total', None)

        with open(filepath, 'wb') as f, tqdm(**progress_bar_params) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
        print(f"\nDescarga de '{filename}' completada y guardada en '{filepath}'.\n")
        return filepath
    except requests.exceptions.RequestException as req_err:
        print(f"Error de red/HTTP durante la descarga de '{filename}': {req_err}")
        if os.path.exists(filepath): os.remove(filepath)
        raise
    except Exception as e:
        print(f"Ocurrió un error inesperado durante la descarga de '{filename}': {e}")
        if os.path.exists(filepath): os.remove(filepath)
        raise

print("Función de descarga definida.")

In [ ]:
# Celda 4: Descarga y Extracción del Shapefile DENUE de Aguascalientes
# ---------------------------------------------------------------------------
CODIGO_ESTADO_DENUE_SHP = "01" # Aguascalientes
NOMBRE_ESTADO_DENUE_SHP = "Aguascalientes"
CRS_DENUE_SHP = "EPSG:4326" # Confirmado por el .prj

# Directorio para datos del DENUE (Shapefile)
DIR_BASE_DENUE_SHP = "./denue_shp_ags" # Nuevo directorio para claridad
DIR_DENUE_SHP_DESCARGAS = os.path.join(DIR_BASE_DENUE_SHP, "descargas_zip")
DIR_DENUE_SHP_EXTRAIDOS = os.path.join(DIR_BASE_DENUE_SHP, "shp_extraidos") # Donde irá el .shp y sus componentes

# Limpieza opcional
if os.path.exists(DIR_BASE_DENUE_SHP):
    # shutil.rmtree(DIR_BASE_DENUE_SHP) # Descomentar para limpiar
    print(f"Directorio {DIR_BASE_DENUE_SHP} ya existe.")
os.makedirs(DIR_DENUE_SHP_DESCARGAS, exist_ok=True)
os.makedirs(DIR_DENUE_SHP_EXTRAIDOS, exist_ok=True)

# URL específica para el SHP del DENUE de Aguascalientes
URL_DENUE_AGS_SHP_ZIP = f"https://www.inegi.org.mx/contenidos/masiva/denue/denue_{CODIGO_ESTADO_DENUE_SHP}_shp.zip"

print(f"Descargando archivo ZIP del DENUE (SHP) para {NOMBRE_ESTADO_DENUE_SHP}...")
zip_denue_shp_path = download(URL_DENUE_AGS_SHP_ZIP, DIR_DENUE_SHP_DESCARGAS)

# Nombres de los archivos componentes del shapefile DENTRO del ZIP
# Basado en tu 'tree': conjunto_de_datos/denue_inegi_01_.shp, etc.
SHP_BASENAME_IN_ZIP = f"denue_inegi_{CODIGO_ESTADO_DENUE_SHP}_" # "denue_inegi_01_"
SHP_COMPONENT_EXTENSIONS = ['.shp', '.shx', '.dbf', '.prj', '.fix', '.qix'] # .cpg no está listado, si se necesita se puede crear

# Ruta de la subcarpeta dentro del ZIP
SUBFOLDER_IN_ZIP_DENUE = "conjunto_de_datos"

# Extraer los componentes del shapefile del DENUE
extracted_shp_denue_mainfile_path = os.path.join(DIR_DENUE_SHP_EXTRAIDOS, f"{SHP_BASENAME_IN_ZIP}.shp")

if not os.path.exists(extracted_shp_denue_mainfile_path) and zip_denue_shp_path:
    print(f"Extrayendo componentes del Shapefile del DENUE de '{zip_denue_shp_path}'...")
    try:
        with ZipFile(zip_denue_shp_path, 'r') as zip_ref:
            for ext in SHP_COMPONENT_EXTENSIONS:
                file_to_extract_in_zip = os.path.join(SUBFOLDER_IN_ZIP_DENUE, f"{SHP_BASENAME_IN_ZIP}{ext}")
                # Extraer directamente al directorio de salida sin la subcarpeta 'conjunto_de_datos'
                # Para esto, necesitamos el nombre base del archivo sin la ruta del zip
                filename_only = os.path.basename(file_to_extract_in_zip)
                target_extraction_path = os.path.join(DIR_DENUE_SHP_EXTRAIDOS, filename_only)

                try:
                    # zip_ref.extract(member, path) extrae 'member' manteniendo su nombre en 'path'
                    # Si queremos renombrar o cambiar estructura, es más complejo o se extrae y mueve.
                    # Manera simple: extraer todo 'conjunto_de_datos' y luego mover los archivos.
                    # O extraer uno por uno a un path temporal y luego mover.

                    # Extraer el miembro a la carpeta destino final.
                    # ZipFile.extract() extrae el miembro con su nombre completo (incluyendo rutas internas del zip)
                    # al directorio especificado.
                    # Necesitamos extraer solo el archivo a DIR_DENUE_SHP_EXTRAIDOS
                    with zip_ref.open(file_to_extract_in_zip) as source_file:
                        with open(target_extraction_path, 'wb') as target_file:
                            shutil.copyfileobj(source_file, target_file)
                    print(f"  Extraído: {filename_only} a {DIR_DENUE_SHP_EXTRAIDOS}")

                except KeyError:
                    print(f"  Advertencia: Archivo '{file_to_extract_in_zip}' no encontrado en el ZIP. Saltando.")
                except Exception as e_file_extract:
                    print(f"  Error extrayendo '{file_to_extract_in_zip}': {e_file_extract}")

            # Verificar si se necesita crear un archivo .cpg (codificación)
            # Si el .dbf tiene acentos, y no hay .cpg, GeoPandas podría usar una codificación incorrecta.
            # El DENUE suele usar 'Windows-1252' o 'ISO-8859-1' para español.
            cpg_path = os.path.join(DIR_DENUE_SHP_EXTRAIDOS, f"{SHP_BASENAME_IN_ZIP}.cpg")
            if not os.path.exists(cpg_path):
                with open(cpg_path, 'w') as f_cpg:
                    f_cpg.write("ISO-8859-1") # O Windows-1252
                print(f"  Archivo .cpg creado por defecto con ISO-8859-1 en: {cpg_path}")

        print(f"Componentes del Shapefile del DENUE extraídos a: {DIR_DENUE_SHP_EXTRAIDOS}")
    except Exception as e_extract_shp:
        print(f"  ¡ERROR durante la extracción del Shapefile del DENUE!: {e_extract_shp}")
        extracted_shp_denue_mainfile_path = None
elif os.path.exists(extracted_shp_denue_mainfile_path):
    print(f"Shapefile del DENUE ya extraído en: {DIR_DENUE_SHP_EXTRAIDOS}")
else:
    print("No se pudo descargar o encontrar el ZIP del Shapefile del DENUE.")
    extracted_shp_denue_mainfile_path = None

print("Proceso de descarga y extracción del Shapefile DENUE completado (o intentado).")

In [ ]:
# Celda 5: Procesar Shapefile del DENUE y Crear GeoDataFrame de Farmacias
# -----------------------------------------------------------------------------------------
# Esta celda depende de 'extracted_shp_denue_mainfile_path' y 'CODIGO_ESTADO_DENUE_SHP'
# y 'CRS_DENUE_SHP' de la Celda 4.

gdf_farmacias_ags = None # Inicializar el GeoDataFrame resultante

# Verificar que el archivo principal del shapefile exista
if 'extracted_shp_denue_mainfile_path' in locals() and \
   extracted_shp_denue_mainfile_path and \
   os.path.exists(extracted_shp_denue_mainfile_path):

    print(f"Procesando Shapefile del DENUE: {extracted_shp_denue_mainfile_path}")

    try:
        # Leer el Shapefile del DENUE.
        # El CRS debería ser EPSG:4326 según el .prj que proporcionaste.
        # La codificación del archivo .dbf es importante para los atributos de texto.
        # 'ISO-8859-1' (latin1) o 'Windows-1252' son comunes para datos en español del INEGI.
        # Si los acentos o caracteres especiales se ven mal, prueba con otra codificación.
        gdf_denue_ags_full = gpd.read_file(extracted_shp_denue_mainfile_path, encoding="ISO-8859-1")

        print(f"Shapefile del DENUE de Aguascalientes cargado con {len(gdf_denue_ags_full)} registros.")
        print(f"CRS original del Shapefile leído por GeoPandas: {gdf_denue_ags_full.crs}")
        print(f"Primeras filas y columnas del Shapefile DENUE completo:")
        print(gdf_denue_ags_full.head(2))
        print(f"Columnas disponibles: {gdf_denue_ags_full.columns.tolist()}")

        # Asegurar/Verificar el CRS (aunque el .prj debería haberlo establecido)
        if gdf_denue_ags_full.crs is None:
            print(f"  ADVERTENCIA: GeoPandas no detectó un CRS. Asignando {CRS_DENUE_SHP} basado en el .prj proporcionado.")
            gdf_denue_ags_full.set_crs(CRS_DENUE_SHP, inplace=True)
        elif gdf_denue_ags_full.crs.to_string().upper() != CRS_DENUE_SHP: # Comparar como string por si hay diferencias menores
             print(f"  ADVERTENCIA: CRS leído ({gdf_denue_ags_full.crs}) no coincide con el esperado ({CRS_DENUE_SHP}). Re-estableciendo a {CRS_DENUE_SHP}.")
             gdf_denue_ags_full = gdf_denue_ags_full.to_crs(CRS_DENUE_SHP) # Usar to_crs si ya tiene uno, set_crs si es None

        # --- Identificación de Columnas Clave ---

        # Columna de Código de Actividad Económica:

        col_codigo_actividad = None
        possible_act_cols = ['COD_ACT', 'cod_act', 'CODIGO_ACT', 'CODIGO_A_1', 'codigo_act']
        for col in possible_act_cols:
            if col in gdf_denue_ags_full.columns:
                col_codigo_actividad = col
                print(f"  Columna de código de actividad identificada como: '{col_codigo_actividad}'")
                break
        if not col_codigo_actividad:
            raise ValueError(f"No se pudo encontrar una columna de código de actividad en {possible_act_cols}. Columnas disponibles: {gdf_denue_ags_full.columns.tolist()}")

        # Columna de Identificador Único del Establecimiento:
        col_id_denue = None
        possible_id_cols = ['id', 'ID', 'CLEE']
        for col in possible_id_cols:
            if col in gdf_denue_ags_full.columns:
                col_id_denue = col
                print(f"  Columna de ID del establecimiento identificada como: '{col_id_denue}'")
                break
        if not col_id_denue:
            print(f"  ADVERTENCIA: No se encontró una columna de ID clara en {possible_id_cols}. Se usará el índice de GeoPandas si es necesario.")
            # Se creará 'id' a partir del índice más adelante si es necesario.

        # Columnas de Nombre del Establecimiento y Nombre de la Actividad (para referencia)
        col_nom_estab = 'NOM_ESTAB' if 'NOM_ESTAB' in gdf_denue_ags_full.columns else ('nom_estab' if 'nom_estab' in gdf_denue_ags_full.columns else None)
        col_nombre_act = 'NOMBRE_ACT' if 'NOMBRE_ACT' in gdf_denue_ags_full.columns else ('nombre_act' if 'nombre_act' in gdf_denue_ags_full.columns else None)

        if not col_nom_estab: print("  Advertencia: Columna 'NOM_ESTAB' no encontrada.")
        if not col_nombre_act: print("  Advertencia: Columna 'NOMBRE_ACT' no encontrada.")


        # --- Filtrado por Actividad Económica (Farmacias) ---
        codigos_farmacias_num = [464111, 464112] # Numéricos

        # El código de actividad en el SHP del DENUE a veces es texto, otras numérico.
        # Convertir a numérico para asegurar la comparación, manejando errores.
        gdf_denue_ags_full[col_codigo_actividad] = pd.to_numeric(gdf_denue_ags_full[col_codigo_actividad], errors='coerce')
        gdf_denue_ags_full.dropna(subset=[col_codigo_actividad], inplace=True) # Eliminar filas donde la conversión falló
        gdf_denue_ags_full[col_codigo_actividad] = gdf_denue_ags_full[col_codigo_actividad].astype(int) # Convertir a entero después de limpiar NaNs

        gdf_farmacias_ags_filtrado = gdf_denue_ags_full[gdf_denue_ags_full[col_codigo_actividad].isin(codigos_farmacias_num)].copy()
        print(f"Farmacias en Aguascalientes (desde SHP, códigos {codigos_farmacias_num}) encontradas: {len(gdf_farmacias_ags_filtrado)}.")

        if not gdf_farmacias_ags_filtrado.empty:
            # --- Selección y Renombrado de Columnas Finales ---
            # El Shapefile ya tiene la columna 'geometry'.

            column_mapping = {}
            if col_id_denue:
                column_mapping[col_id_denue] = 'id' # Estandarizar a 'id'
            if col_nom_estab:
                column_mapping[col_nom_estab] = 'nom_estab'
            if col_nombre_act:
                column_mapping[col_nombre_act] = 'nombre_act'
            column_mapping[col_codigo_actividad] = 'cod_act_num' # Guardar como numérico

            # Crear el GeoDataFrame final solo con las columnas de interés y la geometría
            # Primero, tomar las columnas de atributos que nos interesan
            cols_finales_atributos = []
            if col_id_denue: cols_finales_atributos.append(col_id_denue)
            if col_nom_estab: cols_finales_atributos.append(col_nom_estab)
            if col_nombre_act: cols_finales_atributos.append(col_nombre_act)
            cols_finales_atributos.append(col_codigo_actividad)

            # Asegurar que solo se tomen columnas existentes
            cols_finales_atributos = [c for c in cols_finales_atributos if c in gdf_farmacias_ags_filtrado.columns]

            gdf_farmacias_ags = gdf_farmacias_ags_filtrado[cols_finales_atributos + ['geometry']].copy()

            # Renombrar columnas a nombres estándar
            gdf_farmacias_ags.rename(columns=column_mapping, inplace=True)

            # Si no se encontró una columna de ID, crear una a partir del índice
            if 'id' not in gdf_farmacias_ags.columns:
                print("  Creando columna 'id' a partir del índice.")
                gdf_farmacias_ags['id'] = gdf_farmacias_ags.index.astype(str) # Usar string para el ID


            print("\nGeoDataFrame final de farmacias (gdf_farmacias_ags) preparado:")
            print(gdf_farmacias_ags[['id', 'nom_estab', 'cod_act_num', 'geometry']].head())
            print(f"CRS final de gdf_farmacias_ags: {gdf_farmacias_ags.crs}")

        else:
            print("No se encontraron farmacias con los códigos especificados en el Shapefile de Aguascalientes.")
            gdf_farmacias_ags = gpd.GeoDataFrame(columns=['id', 'nom_estab', 'nombre_act', 'cod_act_num', 'geometry'], crs=CRS_DENUE_SHP) # GDF vacío con estructura

    except Exception as e_proc_shp:
        print(f"Error Crítico al procesar el Shapefile del DENUE: {e_proc_shp}")
        import traceback
        traceback.print_exc()
        gdf_farmacias_ags = None

else:
    print("Shapefile del DENUE no fue extraído o no se encontró en la Celda 4. No se puede continuar con el procesamiento en Celda 5.")
    gdf_farmacias_ags = None # Asegurar que es None si no se procesó

print("\nProcesamiento del Shapefile DENUE y creación de GeoDataFrame de farmacias completado (o intentado).")

In [ ]:
# Celda 6: Conexión a DuckDB e Integración de Farmacias (con INSTALL spatial)
# ------------------------------------------------------------------------------------
# DB_FILE_PATH_EXISTING debe apuntar a tu ./inegi_analisis_01.duckdb

DB_FILE_PATH_EXISTING = "./inegi_analisis_01.duckdb" # Asegúrate que esta sea la ruta correcta
CRS_DB_GEOMETRY = "EPSG:4326" # Todas las geometrías en DB estarán en este CRS

con_integra_denue = None # Inicializar para el bloque finally

if 'gdf_farmacias_ags' in locals() and gdf_farmacias_ags is not None and not gdf_farmacias_ags.empty:
    if os.path.exists(DB_FILE_PATH_EXISTING):
        print(f"Conectando a la base de datos existente: {DB_FILE_PATH_EXISTING}")
        try:
            con_integra_denue = duckdb.connect(database=DB_FILE_PATH_EXISTING, read_only=False) # Necesitamos escribir

            print("Instalando (si es necesario) y cargando la extensión 'spatial'...")
            con_integra_denue.execute("INSTALL spatial;")
            con_integra_denue.execute("LOAD spatial;")
            print("Conexión establecida y extensión espacial 'spatial' lista.")

            TABLE_NAME_FARMACIAS = "denue_farmacias_ags"
            con_integra_denue.execute(f"DROP TABLE IF EXISTS {TABLE_NAME_FARMACIAS};")

            gdf_farmacias_to_db = gdf_farmacias_ags.copy()

            if gdf_farmacias_to_db.crs is None or gdf_farmacias_to_db.crs.to_string().upper() != CRS_DB_GEOMETRY:
                print(f"  Asegurando CRS de gdf_farmacias_to_db a {CRS_DB_GEOMETRY}...")
                gdf_farmacias_to_db = gdf_farmacias_to_db.to_crs(CRS_DB_GEOMETRY)

            gdf_farmacias_to_db['geom_wkt'] = gdf_farmacias_to_db['geometry'].apply(lambda g: g.wkt if g and not g.is_empty else None)

            col_codigo_actividad_en_gdf = 'cod_act_num' if 'cod_act_num' in gdf_farmacias_to_db.columns else ('cod_act' if 'cod_act' in gdf_farmacias_to_db.columns else None)

            cols_to_register = ['id', 'nom_estab', 'geom_wkt']
            if 'nombre_act' in gdf_farmacias_to_db.columns: cols_to_register.append('nombre_act')
            if col_codigo_actividad_en_gdf: cols_to_register.append(col_codigo_actividad_en_gdf)

            cols_existentes_in_gdf = [col for col in cols_to_register if col in gdf_farmacias_to_db.columns]
            if 'geom_wkt' not in cols_existentes_in_gdf:
                 cols_existentes_in_gdf.append('geom_wkt')

            df_for_duckdb_reg = gdf_farmacias_to_db[cols_existentes_in_gdf].copy()
            df_for_duckdb_reg.dropna(subset=['geom_wkt'], inplace=True)

            if df_for_duckdb_reg.empty:
                print("  ADVERTENCIA: No hay datos de farmacias con geometría válida para cargar en DuckDB.")
            else:
                con_integra_denue.register('df_farmacias_temp_reg', df_for_duckdb_reg)

                # Crear la definición de columnas para la tabla, renombrando col_codigo_actividad_en_gdf a cod_act si es necesario para la DB
                # y asegurando que los tipos sean inferidos correctamente por DuckDB desde el DataFrame,
                # excepto para la geometría que se crea con ST_GeomFromText.

                # Generar lista de columnas para el SELECT, renombrando si es necesario
                select_cols_for_create = []
                for col in df_for_duckdb_reg.columns:
                    if col == 'geom_wkt':
                        continue # Se maneja con ST_GeomFromText
                    elif col == col_codigo_actividad_en_gdf and col_codigo_actividad_en_gdf != 'cod_act':
                        select_cols_for_create.append(f'"{col}" AS cod_act') # Renombrar a un estándar 'cod_act' en DB
                    else:
                        select_cols_for_create.append(f'"{col}"')


                con_integra_denue.execute(f"""
                    CREATE TABLE {TABLE_NAME_FARMACIAS} AS
                    SELECT
                        {', '.join(select_cols_for_create)},
                        ST_GeomFromText(geom_wkt) AS geometry_db
                    FROM df_farmacias_temp_reg;
                """)

                num_farmacias_db = con_integra_denue.execute(f"SELECT COUNT(*) FROM {TABLE_NAME_FARMACIAS};").fetchone()[0]
                print(f"Tabla '{TABLE_NAME_FARMACIAS}' creada/reemplazada en DuckDB con {num_farmacias_db} registros.")
                print("Primeras filas de la tabla de farmacias en DuckDB:")

                cols_in_db_farmacias = [desc[0] for desc in con_integra_denue.execute(f"DESCRIBE {TABLE_NAME_FARMACIAS};").fetchall()]
                select_preview_cols = ['id', 'nom_estab']
                if 'cod_act' in cols_in_db_farmacias: select_preview_cols.append('cod_act') # Si se renombró a cod_act
                elif col_codigo_actividad_en_gdf in cols_in_db_farmacias : select_preview_cols.append(col_codigo_actividad_en_gdf)


                print(con_integra_denue.execute(f"SELECT {', '.join(select_preview_cols)} FROM {TABLE_NAME_FARMACIAS} LIMIT 3;").fetchdf())

        except Exception as e_db_farm:
            print(f"Error al integrar farmacias en DuckDB: {e_db_farm}")
            import traceback
            traceback.print_exc()
        # No cerramos la conexión con_integra_denue aquí, se usará en la Celda 7.

    else:
        print(f"¡ERROR! La base de datos '{DB_FILE_PATH_EXISTING}' no fue encontrada.")
else:
    print("No hay GeoDataFrame 'gdf_farmacias_ags' para cargar en DuckDB. Ejecuta la Celda 5 primero.")

print("\nIntegración de farmacias (desde SHP) en DuckDB completada (o intentada).")

In [ ]:
# Celda 7: Análisis Espacial en DuckDB - Conteo de Farmacias por Manzana
# -----------------------------------------------------------------------------------

TABLE_NAME_FARMACIAS = "denue_farmacias_ags"

# Primero, asegurémonos de que con_integra_denue existe y está asignada.
if 'con_integra_denue' not in locals() or con_integra_denue is None:
    print("La conexión 'con_integra_denue' no fue establecida en la celda anterior. Intentando reconectar...")
    DB_FILE_PATH_EXISTING_C7 = "./inegi_analisis_01.duckdb" # Asegúrate que esta sea la ruta correcta
    if os.path.exists(DB_FILE_PATH_EXISTING_C7):
        try:
            con_integra_denue = duckdb.connect(database=DB_FILE_PATH_EXISTING_C7, read_only=False)
            print("Instalando (si es necesario) y cargando la extensión 'spatial' en reconexión...")
            con_integra_denue.execute("INSTALL spatial;")
            con_integra_denue.execute("LOAD spatial;")
            print("Reconexión exitosa y extensión 'spatial' lista.")
        except Exception as e_reconnect:
            print(f"Fallo al reconectar o cargar extensión spatial: {e_reconnect}")
            con_integra_denue = None
    else:
        print(f"Archivo de base de datos {DB_FILE_PATH_EXISTING_C7} no encontrado para reconexión.")
        con_integra_denue = None


if con_integra_denue: # Proceder solo si la conexión es válida
    print("\nRealizando conteo de farmacias por manzana en DuckDB...")

    try:
        # Verificar si las tablas 'censo_geo' y 'denue_farmacias_ags' existen
        censo_geo_exists = con_integra_denue.execute("SELECT 1 FROM information_schema.tables WHERE table_name = 'censo_geo'").fetchone()
        denue_farmacias_exists = con_integra_denue.execute(f"SELECT 1 FROM information_schema.tables WHERE table_name = '{TABLE_NAME_FARMACIAS}'").fetchone()

        if censo_geo_exists and denue_farmacias_exists:
            TABLE_MANZANAS_CON_FARMACIAS = "manzanas_con_conteo_farmacias_ags"

            con_integra_denue.execute(f"DROP TABLE IF EXISTS {TABLE_MANZANAS_CON_FARMACIAS};")

            query_conteo = f"""
            CREATE TABLE {TABLE_MANZANAS_CON_FARMACIAS} AS
            SELECT
                m.*,
                COALESCE(f.num_farmacias, 0) AS num_farmacias_en_manzana
            FROM censo_geo m
            LEFT JOIN (
                SELECT
                    m_join.CVEGEO AS CVEGEO_manzana,
                    COUNT(denue.id) AS num_farmacias
                FROM censo_geo m_join
                JOIN {TABLE_NAME_FARMACIAS} denue
                    ON ST_Intersects(denue.geometry_db, m_join.geometry)
                GROUP BY m_join.CVEGEO
            ) f ON m.CVEGEO = f.CVEGEO_manzana;
            """

            con_integra_denue.execute(query_conteo)
            num_manzanas_final = con_integra_denue.execute(f"SELECT COUNT(*) FROM {TABLE_MANZANAS_CON_FARMACIAS}").fetchone()[0]
            sum_farmacias_contadas_result = con_integra_denue.execute(f"SELECT SUM(num_farmacias_en_manzana) FROM {TABLE_MANZANAS_CON_FARMACIAS}").fetchone()
            sum_farmacias_contadas = sum_farmacias_contadas_result[0] if sum_farmacias_contadas_result else 0

            print(f"Tabla '{TABLE_MANZANAS_CON_FARMACIAS}' creada con {num_manzanas_final} manzanas.")
            print(f"Total de 'ubicaciones' de farmacias contadas y asignadas a manzanas: {sum_farmacias_contadas if sum_farmacias_contadas else 0}")

            print("\nPrimeras filas de la tabla con conteo de farmacias:")
            df_preview_conteo = con_integra_denue.execute(f"SELECT CVEGEO, POBTOT, NOM_MUN, num_farmacias_en_manzana FROM {TABLE_MANZANAS_CON_FARMACIAS} WHERE num_farmacias_en_manzana > 0 LIMIT 5").fetchdf()
            if not df_preview_conteo.empty:
                print(df_preview_conteo)
            else:
                df_preview_conteo_general = con_integra_denue.execute(f"SELECT CVEGEO, POBTOT, NOM_MUN, num_farmacias_en_manzana FROM {TABLE_MANZANAS_CON_FARMACIAS} LIMIT 5").fetchdf()
                print(df_preview_conteo_general)
                print(" (Ninguna de las primeras manzanas en la vista previa tenía farmacias).")

        else: # Si alguna tabla no existe
            if not censo_geo_exists: print("¡ERROR! La tabla 'censo_geo' no existe.")
            if not denue_farmacias_exists: print(f"¡ERROR! La tabla '{TABLE_NAME_FARMACIAS}' no existe.")
            print("No se puede realizar el conteo.")

    except duckdb.Error as e_duck_c7:
        print(f"Error de DuckDB en Celda 7: {e_duck_c7}")
        import traceback; traceback.print_exc()
    except Exception as e_general_c7:
        print(f"Error general en Celda 7: {e_general_c7}")
        import traceback; traceback.print_exc()
    finally:

        if 'con_integra_denue' in locals() and con_integra_denue:
            try:
                con_integra_denue.close()
                print("\nConexión 'con_integra_denue' cerrada al finalizar Celda 7.")
            except Exception as e_close_c7:
                print(f"Error al cerrar 'con_integra_denue' en Celda 7: {e_close_c7}")
        elif 'con_integra_denue' not in locals() or con_integra_denue is None:
             print("\nAnálisis espacial en Celda 7 completado (o intentado), la conexión no estaba activa o no se reestableció.")



print("\nAnálisis espacial (conteo de farmacias desde SHP) completado (o intentado).")

In [ ]:
# Celda 8: Exportar Resultado a GeoPackage y Descargar
# ------------------------------------------------------------------------------------

import duckdb
import geopandas as gpd
import pandas as pd
from shapely import wkt # Para leer y escribir WKT
# from shapely import wkb # No se usa directamente si ST_AsText es la fuente
import os
from IPython.display import display # Para Colab si se usa files.download

print("--- Iniciando Exportación a GeoPackage (Corregido KeyError) ---")

DB_FILE_PATH_FINAL_DENUE_SHP = "./inegi_analisis_01.duckdb"
TABLE_TO_EXPORT_DENUE_SHP = "manzanas_con_conteo_farmacias_ags"

DIR_BASE_DENUE_SHP_EXPORT = "./denue_shp_ags"
GPKG_OUTPUT_DIR_DENUE_SHP = os.path.join(DIR_BASE_DENUE_SHP_EXPORT, "geopackages_denue_shp_output")
os.makedirs(GPKG_OUTPUT_DIR_DENUE_SHP, exist_ok=True)

GPKG_FILENAME_DENUE_SHP = f"manzanas_ags_con_farmacias_denue_shp.gpkg"
GPKG_FILE_PATH_DENUE_SHP = os.path.join(GPKG_OUTPUT_DIR_DENUE_SHP, GPKG_FILENAME_DENUE_SHP)
LAYER_NAME_DENUE_SHP_GPKG = "manzanas_con_farmacias_shp"
CRS_EXPORT = "EPSG:4326"

con_export_denue_shp = None

print(f"\nIntentando exportar la tabla '{TABLE_TO_EXPORT_DENUE_SHP}' a GeoPackage...")

if os.path.exists(DB_FILE_PATH_FINAL_DENUE_SHP):
    try:
        con_export_denue_shp = duckdb.connect(database=DB_FILE_PATH_FINAL_DENUE_SHP, read_only=True)
        print("Instalando (si es necesario) y cargando la extensión 'spatial' para exportación...")
        con_export_denue_shp.execute("INSTALL spatial;")
        con_export_denue_shp.execute("LOAD spatial;")
        print(f"Conexión a '{DB_FILE_PATH_FINAL_DENUE_SHP}' establecida (read_only) y extensión 'spatial' lista para exportación.")

        table_exists_check = con_export_denue_shp.execute(f"SELECT 1 FROM information_schema.tables WHERE table_name = '{TABLE_TO_EXPORT_DENUE_SHP}'").fetchone()
        if not table_exists_check:
            print(f"¡ERROR! La tabla '{TABLE_TO_EXPORT_DENUE_SHP}' no existe. No se puede exportar.")
        else:
            print(f"Leyendo datos de '{TABLE_TO_EXPORT_DENUE_SHP}'...")

            query_export = f"SELECT *, ST_AsText(geometry) AS geom_wkt FROM {TABLE_TO_EXPORT_DENUE_SHP} WHERE geometry IS NOT NULL;"
            df_to_export_duck = con_export_denue_shp.execute(query_export).fetch_df()

            if df_to_export_duck.empty:
                print(f"La tabla '{TABLE_TO_EXPORT_DENUE_SHP}' está vacía o sin geometrías válidas. No se generará GeoPackage.")
            else:
                print(f"  Datos leídos. {len(df_to_export_duck)} filas. Procesando geometrías WKT...")

                def parse_wkt_for_export(wkt_string):
                    if wkt_string is None or not isinstance(wkt_string, str) or wkt_string.strip() == "": return None
                    try: return wkt.loads(wkt_string)
                    except: return None

                df_to_export_duck['geometry_parsed'] = df_to_export_duck['geom_wkt'].apply(parse_wkt_for_export)

                cols_to_drop = ['geom_wkt']
                if 'geometry' in df_to_export_duck.columns:
                    cols_to_drop.append('geometry')

                gdf_to_export = gpd.GeoDataFrame(
                    df_to_export_duck.drop(columns=cols_to_drop, errors='ignore'),
                    geometry='geometry_parsed',  # La columna de geometría activa es 'geometry_parsed'
                    crs=CRS_EXPORT
                )


                gdf_to_export.dropna(subset=['geometry_parsed'], inplace=True)


                gdf_to_export.rename_geometry('geometry', inplace=True)

                if gdf_to_export.empty:
                    print("GeoDataFrame para exportar está vacío después de procesar geometrías WKT o eliminar NAs.")
                else:
                    print(f"GeoDataFrame listo con {len(gdf_to_export)} filas. Guardando en '{GPKG_FILE_PATH_DENUE_SHP}'...")
                    gdf_to_export.to_file(GPKG_FILE_PATH_DENUE_SHP, layer=LAYER_NAME_DENUE_SHP_GPKG, driver="GPKG")
                    print("¡GeoPackage (desde DENUE SHP) guardado exitosamente!")

                    from google.colab import files
                    print(f"\nSolicitando descarga de: {GPKG_FILE_PATH_DENUE_SHP}")
                    files.download(GPKG_FILE_PATH_DENUE_SHP)
    except Exception as e_export:
        print(f"Error durante la exportación a GeoPackage: {e_export}")
        import traceback; traceback.print_exc()
    finally:
        if con_export_denue_shp:
            try: con_export_denue_shp.close(); print("Conexión de exportación cerrada.")
            except: pass
else:
    print(f"La base de datos '{DB_FILE_PATH_FINAL_DENUE_SHP}' no fue encontrada para la exportación.")

print("\n--- Fin del Script de Integración DENUE (SHP) y Exportación (Corregido KeyError) ---")

In [ ]:
# Celda 9: Instalación de OSMnx
# ------------------------------------------------------------------------
# Esta celda se puede omitir si ya ejecutaste una similar en la sesión.
!pip install osmnx matplotlib networkx --quiet
print("OSMnx y dependencias instaladas/verificadas.")

In [ ]:
# Celda 10: Importaciones Adicionales para Isócronas
# --------------------------------------------------
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import unary_union
import geopandas as gpd
import duckdb
from shapely import wkt, wkb # Para leer/escribir geometrías

print("Librerías para análisis de isócronas importadas/verificadas.")

In [ ]:
# Celda 11: Configuración y Carga de Polígono de Localidad para la Red Vial
# ------------------------------------------------------------------------------------

CODIGO_ESTADO_ISO_MZA = "01"
NOMBRE_ESTADO_ISO_MZA = "Aguascalientes"
CRS_GEOGRAFICO_ISO_MZA = "EPSG:4326"
CRS_PROYECTADO_ISO_MZA = "EPSG:6372"

DB_FILE_PATH_ISO_MZA = "./inegi_analisis_01.duckdb" # La base de datos principal

# Directorio para salidas de estas nuevas isócronas
DIR_BASE_ISOCRONAS_MZA = "./isocronas_manzanas_ags" # Nuevo directorio
os.makedirs(DIR_BASE_ISOCRONAS_MZA, exist_ok=True)
GPKG_ISOCRONAS_MZA_OUTPUT_DIR = os.path.join(DIR_BASE_ISOCRONAS_MZA, "geopackages_isocronas_manzanas_output")
os.makedirs(GPKG_ISOCRONAS_MZA_OUTPUT_DIR, exist_ok=True)

con_loc_poly = None
gdf_localidad_ags_para_red = None # Cambiado el nombre para claridad

try:
    print(f"Conectando a DuckDB: {DB_FILE_PATH_ISO_MZA} para obtener polígono de localidad.")
    con_loc_poly = duckdb.connect(database=DB_FILE_PATH_ISO_MZA, read_only=True)
    con_loc_poly.execute("LOAD spatial;")
    print("Conexión establecida y extensión espacial cargada.")

    # Reutilizar la lógica de BBOX de farmacias o polígono por defecto si no hay capa de localidades
    # Esto es solo para definir el área de la red vial de OSMnx
    farmacias_bbox_q = """
    SELECT
        MIN(ST_X(geometry_db)) as minx, MIN(ST_Y(geometry_db)) as miny,
        MAX(ST_X(geometry_db)) as maxx, MAX(ST_Y(geometry_db)) as maxy
    FROM denue_farmacias_ags;
    """ # denue_farmacias_ags debe existir

    denue_exists_check = con_loc_poly.execute("SELECT 1 FROM information_schema.tables WHERE table_name = 'denue_farmacias_ags'").fetchone()
    if not denue_exists_check:
        raise ValueError("La tabla 'denue_farmacias_ags' es necesaria para definir el área de la red vial (usando su BBOX). Ejecute las celdas de procesamiento del DENUE primero.")

    bbox_res_loc = con_loc_poly.execute(farmacias_bbox_q).fetchone()

    if bbox_res_loc and all(bbox_res_loc):
        minx, miny, maxx, maxy = bbox_res_loc
        # Expandir un poco el BBOX para asegurar cobertura
        buffer_deg = 0.01
        ags_polygon_boundary_loc = Polygon([
            (minx-buffer_deg, miny-buffer_deg), (minx-buffer_deg, maxy+buffer_deg),
            (maxx+buffer_deg, maxy+buffer_deg), (maxx+buffer_deg, miny-buffer_deg),
            (minx-buffer_deg, miny-buffer_deg)
        ])
        gdf_localidad_ags_para_red = gpd.GeoDataFrame(
            [{'id': 1, 'nombre': 'Area Aguascalientes (BBOX Farmacias)', 'geometry': ags_polygon_boundary_loc}],
            crs=CRS_GEOGRAFICO_ISO_MZA
        )
        print("Polígono para red vial definido (aproximado por BBOX de farmacias expandido).")
    else:
        print("ADVERTENCIA: No se pudo obtener BBOX de farmacias. Usando polígono de Aguascalientes por defecto.")
        default_poly_ags_loc = Polygon([(-102.4, 21.7), (-102.4, 22.0), (-102.15, 22.0), (-102.15, 21.7), (-102.4, 21.7)]) # Ampliado
        gdf_localidad_ags_para_red = gpd.GeoDataFrame(
            [{'id':1, 'nombre': 'Area Aguascalientes (Default)', 'geometry': default_poly_ags_loc}],
            crs=CRS_GEOGRAFICO_ISO_MZA
        )
except Exception as e:
    print(f"Error al obtener polígono de localidad para la red: {e}")
    gdf_localidad_ags_para_red = None
finally:
    if con_loc_poly: con_loc_poly.close()

if gdf_localidad_ags_para_red is None or gdf_localidad_ags_para_red.empty:
    raise ValueError("No se pudo definir el polígono del área para descargar la red vial.")

aguascalientes_city_polygon_for_net = gdf_localidad_ags_para_red['geometry'].iloc[0]
print("\nPolígono para descarga de red vial listo.")

# Descargar Red Vial y Preparar Grafo con OSMnx (similar a la celda 12 anterior)
G_mza_iso = None
G_proj_mza_iso = None

if aguascalientes_city_polygon_for_net:
    print("Descargando red vial para 'walk' (caminata) usando OSMnx...")
    try:
        G_mza_iso = ox.graph_from_polygon(aguascalientes_city_polygon_for_net, network_type="walk", simplify=True, retain_all=False)
        print(f"Grafo descargado con {len(G_mza_iso.nodes)} nodos y {len(G_mza_iso.edges)} aristas.")

        G_proj_mza_iso = ox.project_graph(G_mza_iso, to_crs=CRS_PROYECTADO_ISO_MZA)

        walk_speed_kmh_mza = 4.5
        meters_per_minute_mza = walk_speed_kmh_mza * 1000 / 60

        for u, v, data in G_proj_mza_iso.edges(data=True):
            data['time_min'] = data['length'] / meters_per_minute_mza

        print("Atributo 'time_min' añadido a las aristas del grafo proyectado.")
    except Exception as e_osmnx_mza:
        print(f"Error durante la descarga o procesamiento de la red vial con OSMnx: {e_osmnx_mza}")
else:
    print("No hay polígono de la ciudad para descargar la red vial.")

if G_proj_mza_iso is None:
    raise ValueError("Fallo al crear el grafo proyectado con tiempos de viaje. No se puede continuar.")

In [ ]:
# Celda 12
# --------
gdf_isocronas_manzanas = None

if 'G_proj_mza_iso' in locals() and G_proj_mza_iso: # Solo si el grafo está listo
    con_mza_farm = None
    try:
        print(f"\nConectando a DuckDB: {DB_FILE_PATH_ISO_MZA} para leer manzanas con farmacias.")
        con_mza_farm = duckdb.connect(database=DB_FILE_PATH_ISO_MZA, read_only=True)
        con_mza_farm.execute("LOAD spatial;")

        TABLE_MANZANAS_CON_FARMACIAS_INPUT = "manzanas_con_conteo_farmacias_ags"

        mza_farm_exists = con_mza_farm.execute(f"SELECT 1 FROM information_schema.tables WHERE table_name = '{TABLE_MANZANAS_CON_FARMACIAS_INPUT}'").fetchone()
        if not mza_farm_exists:
            raise ValueError(f"La tabla '{TABLE_MANZANAS_CON_FARMACIAS_INPUT}' no existe. Ejecute la Celda 7 primero.")

        manzanas_con_farmacias_query = f"""
        SELECT
            CVEGEO,
            NOM_MUN,
            POBTOT,
            num_farmacias_en_manzana,
            ST_AsText(geometry) as geom_wkt_manzana -- Geometría del POLÍGONO de la manzana como WKT
        FROM {TABLE_MANZANAS_CON_FARMACIAS_INPUT}
        WHERE num_farmacias_en_manzana > 0 AND geometry IS NOT NULL AND ST_IsValid(geometry);
        """
        df_manzanas_con_farm_db = con_mza_farm.execute(manzanas_con_farmacias_query).fetch_df()

        if df_manzanas_con_farm_db.empty:
            raise ValueError("No se encontraron manzanas con farmacias en la base de datos o no tienen geometrías válidas.")

        def parse_wkt_manzana(wkt_string):
            if wkt_string is None or not isinstance(wkt_string, str) or wkt_string.strip() == "": return None
            try: return wkt.loads(wkt_string)
            except Exception: return None

        df_manzanas_con_farm_db['geometry'] = df_manzanas_con_farm_db['geom_wkt_manzana'].apply(parse_wkt_manzana)

        gdf_manzanas_con_farm = gpd.GeoDataFrame(
            df_manzanas_con_farm_db.drop(columns=['geom_wkt_manzana']),
            geometry='geometry', crs=CRS_GEOGRAFICO_ISO_MZA # Asumiendo que las geometrías en DB están en 4326
        )
        gdf_manzanas_con_farm.dropna(subset=['geometry'], inplace=True) # Eliminar si el parseo WKT falló

        if gdf_manzanas_con_farm.empty:
            raise ValueError("GeoDataFrame de manzanas con farmacias está vacío después del parseo WKT.")

        # Calcular el CENTROIDE de estas manzanas
        # Asegurarse que el GDF tenga un CRS proyectado para un cálculo de centroide más preciso si fuera necesario,
        # pero para nearest_nodes, el centroide en 4326 está bien si el grafo G_mza_iso está en 4326.
        if gdf_manzanas_con_farm.crs.to_epsg() != 4326:
            gdf_manzanas_con_farm = gdf_manzanas_con_farm.to_crs(epsg=4326)

        gdf_manzanas_con_farm['centroid_manzana'] = gdf_manzanas_con_farm.geometry.centroid
        gdf_manzanas_con_farm.dropna(subset=['centroid_manzana'], inplace=True) # Eliminar si el centroide falló

        if gdf_manzanas_con_farm.empty:
            raise ValueError("GeoDataFrame de manzanas con farmacias está vacío después de calcular centroides.")

        print(f"\nGenerando isócronas de 10 minutos a pie para los CENTROIDES de {len(gdf_manzanas_con_farm)} manzanas con farmacias...")

        isocronas_manzanas_polys = []
        manzanas_isocronas_cvegeo = []
        trip_time_max_min_mza = 10

        for index, manzana in tqdm(gdf_manzanas_con_farm.iterrows(), total=len(gdf_manzanas_con_farm), desc="Calculando Isócronas de Manzanas"):
            try:
                centroide_manzana_geom = manzana['centroid_manzana']
                if not centroide_manzana_geom or centroide_manzana_geom.is_empty:
                    isocronas_manzanas_polys.append(None)
                    manzanas_isocronas_cvegeo.append(manzana['CVEGEO'])
                    continue


                center_node_orig_mza = ox.nearest_nodes(G_mza_iso, X=centroide_manzana_geom.x, Y=centroide_manzana_geom.y)
                center_node_proj_mza = center_node_orig_mza # Asumimos que los IDs de nodos se mantienen

                subgraph_mza = nx.ego_graph(G_proj_mza_iso, center_node_proj_mza, radius=trip_time_max_min_mza, distance='time_min')

                if subgraph_mza.number_of_nodes() > 2:
                    node_points_mza = [Point(data['x'], data['y']) for node, data in subgraph_mza.nodes(data=True)]
                    bounding_poly_mza = gpd.GeoSeries(node_points_mza, crs=CRS_PROYECTADO_ISO_MZA).unary_union.convex_hull
                    isocronas_manzanas_polys.append(bounding_poly_mza)
                else:
                    isocronas_manzanas_polys.append(None)
                manzanas_isocronas_cvegeo.append(manzana['CVEGEO'])

            except Exception as e_iso_mza_single:
                print(f"Error generando isócrona para manzana CVEGEO {manzana.get('CVEGEO', 'Desconocido')}: {e_iso_mza_single}")
                isocronas_manzanas_polys.append(None)
                manzanas_isocronas_cvegeo.append(manzana.get('CVEGEO', 'Desconocido'))

        gdf_isocronas_manzanas = gpd.GeoDataFrame(
            {'CVEGEO_manzana': manzanas_isocronas_cvegeo, 'geometry': isocronas_manzanas_polys},
            crs=CRS_PROYECTADO_ISO_MZA # Las isócronas se generan en el CRS del grafo proyectado
        )
        gdf_isocronas_manzanas.dropna(subset=['geometry'], inplace=True)
        gdf_isocronas_manzanas = gdf_isocronas_manzanas.to_crs(CRS_GEOGRAFICO_ISO_MZA) # Convertir de nuevo a 4326

        print(f"\n{len(gdf_isocronas_manzanas)} isócronas de manzanas (con farmacias) generadas.")

    except ValueError as ve: # Capturar ValueErrors que podamos haber lanzado
        print(f"Error de Valor en Celda 12: {ve}")
        gdf_isocronas_manzanas = None
    except duckdb.Error as de: # Capturar errores de DuckDB
        print(f"Error de DuckDB en Celda 12: {de}")
        import traceback; traceback.print_exc()
        gdf_isocronas_manzanas = None
    except Exception as e:
        print(f"Error general en el proceso de generación de isócronas de manzanas (Celda 12): {e}")
        import traceback; traceback.print_exc()
        gdf_isocronas_manzanas = None
    finally:
        if con_mza_farm: con_mza_farm.close()
else:
    print("El grafo de calles (G_proj_mza_iso) no fue generado en Celda 11. No se pueden calcular isócronas de manzanas.")

In [ ]:
# Celda 13 : Guardar Isócronas de Manzanas y Cargar a DuckDB
# ----------------------------------------------------------------------------
if gdf_isocronas_manzanas is not None and not gdf_isocronas_manzanas.empty:
    # Guardar las isócronas de manzanas en un GeoPackage
    isocronas_manzanas_gpkg_path = os.path.join(GPKG_ISOCRONAS_MZA_OUTPUT_DIR, f"isocronas_10min_manzanas_con_farmacias_{CODIGO_ESTADO_ISO_MZA}.gpkg")
    gdf_isocronas_manzanas.to_file(isocronas_manzanas_gpkg_path, layer="isocronas_manzanas_10min", driver="GPKG")
    print(f"Isócronas de manzanas guardadas en: {isocronas_manzanas_gpkg_path}")

    # Cargar estas isócronas a una nueva tabla en DuckDB
    con_iso_mza_db = None
    try:
        print(f"\nConectando a DuckDB: {DB_FILE_PATH_ISO_MZA} para cargar isócronas de manzanas.")
        con_iso_mza_db = duckdb.connect(database=DB_FILE_PATH_ISO_MZA, read_only=False)
        con_iso_mza_db.execute("LOAD spatial;")

        TABLE_NAME_ISOCRONAS_MANZANAS = "isocronas_manzanas_farmacias_ags"
        con_iso_mza_db.execute(f"DROP TABLE IF EXISTS {TABLE_NAME_ISOCRONAS_MANZANAS};")

        gdf_isocronas_manzanas_db = gdf_isocronas_manzanas.copy()
        gdf_isocronas_manzanas_db['geom_wkt'] = gdf_isocronas_manzanas_db['geometry'].apply(lambda g: g.wkt if g and not g.is_empty else None)

        cols_iso_mza_db = [col for col in gdf_isocronas_manzanas_db.columns if col != 'geometry']

        con_iso_mza_db.register('df_isocronas_mza_temp', gdf_isocronas_manzanas_db[cols_iso_mza_db])
        con_iso_mza_db.execute(f"""
            CREATE TABLE {TABLE_NAME_ISOCRONAS_MANZANAS} AS
            SELECT *, ST_GeomFromText(geom_wkt) AS geometry_db
            FROM df_isocronas_mza_temp WHERE geom_wkt IS NOT NULL;
        """)
        # Opcional: con_iso_mza_db.execute(f"ALTER TABLE {TABLE_NAME_ISOCRONAS_MANZANAS} DROP geom_wkt;")
        print(f"Tabla '{TABLE_NAME_ISOCRONAS_MANZANAS}' creada en DuckDB.")

        # Exportación final (o en una celda separada)
        # Aquí podríamos generar un GPKG que una censo_geo con estas nuevas isócronas si es necesario
        # Por ahora, solo hemos guardado las isócronas por separado y en DB.

        # Descargar el GeoPackage de las isócronas de manzanas
        from google.colab import files
        print(f"\nSolicitando descarga de (isócronas de manzanas): {isocronas_manzanas_gpkg_path}")
        files.download(isocronas_manzanas_gpkg_path)

    except Exception as e_db_iso_mza:
        print(f"Error al cargar isócronas de manzanas a DuckDB o exportar: {e_db_iso_mza}")
        import traceback; traceback.print_exc()
    finally:
        if con_iso_mza_db: con_iso_mza_db.close()
else:
    print("No se generaron isócronas de manzanas o el GeoDataFrame está vacío. No se guardó nada.")

print("\n--- Fin del Script de Generación de Isócronas para Manzanas con Farmacias ---")

In [ ]:
# Celda 14: Cruce Espacial - Contar Influencia de Isócronas de Manzanas con Farmacias por Manzana
# ---------------------------------------------------------------------------------------------------------
# Esta celda calcula, para cada manzana, cuántas isócronas de 10 min (generadas desde
# los centroides de manzanas que tienen farmacias) la intersectan.

print("--- Iniciando Cruce: Influencia de Isócronas de Manzanas con Farmacias por Manzana ---")

DB_FILE_PATH_CRUCE = "./inegi_analisis_01.duckdb" # La base de datos principal
TABLE_MANZANAS_BASE = "manzanas_con_conteo_farmacias_ags" # Tabla de manzanas con conteo de farmacias DENTRO
TABLE_ISOCRONAS_MANZANAS_INPUT = "isocronas_manzanas_farmacias_ags" # Tabla de isócronas generadas en Celda 13
TABLE_OUTPUT_CRUCE = "manzanas_con_influencia_isocronas_final_ags"

con_cruce = None

try:
    print(f"Conectando a DuckDB: {DB_FILE_PATH_CRUCE}")
    con_cruce = duckdb.connect(database=DB_FILE_PATH_CRUCE, read_only=False) # Necesitamos escribir la tabla final
    con_cruce.execute("LOAD spatial;")
    print("Conexión establecida y extensión espacial cargada.")

    # Verificar que las tablas de entrada existan
    manzanas_base_exists = con_cruce.execute(f"SELECT 1 FROM information_schema.tables WHERE table_name = '{TABLE_MANZANAS_BASE}'").fetchone()
    isocronas_mza_exists = con_cruce.execute(f"SELECT 1 FROM information_schema.tables WHERE table_name = '{TABLE_ISOCRONAS_MANZANAS_INPUT}'").fetchone()

    if not (manzanas_base_exists and isocronas_mza_exists):
        missing_tables = []
        if not manzanas_base_exists: missing_tables.append(TABLE_MANZANAS_BASE)
        if not isocronas_mza_exists: missing_tables.append(TABLE_ISOCRONAS_MANZANAS_INPUT)
        raise ValueError(f"Las tablas de entrada necesarias ({', '.join(missing_tables)}) no existen. Ejecute las celdas anteriores.")

    con_cruce.execute(f"DROP TABLE IF EXISTS {TABLE_OUTPUT_CRUCE};")

    # La tabla 'isocronas_manzanas_farmacias_ags' tiene 'CVEGEO_manzana' (de la manzana origen de la isócrona)
    # y 'geometry_db' (la isócrona).
    # La tabla 'manzanas_con_conteo_farmacias_ags' (o 'censo_geo') tiene 'CVEGEO' y 'geometry' (polígono de la manzana).
    # Ambas geometrías deberían estar en EPSG:4326.

    query_cruce_influencia = f"""
    CREATE TABLE {TABLE_OUTPUT_CRUCE} AS
    SELECT
        m.*, -- Tomar todas las columnas de la tabla de manzanas base
        COALESCE(iso_influence.num_isocronas_intersectantes, 0) AS influencia_iso_farm_10min
    FROM {TABLE_MANZANAS_BASE} m
    LEFT JOIN (
        SELECT
            m_target.CVEGEO AS CVEGEO_manzana_objetivo,
            COUNT(DISTINCT iso.CVEGEO_manzana) AS num_isocronas_intersectantes -- Contar las isócronas únicas (por su manzana de origen)
        FROM {TABLE_MANZANAS_BASE} m_target -- Cada manzana de la ciudad
        JOIN {TABLE_ISOCRONAS_MANZANAS_INPUT} iso -- Cada isócrona de manzana con farmacia
            ON ST_Intersects(m_target.geometry, iso.geometry_db) -- Si la manzana se intersecta con la isócrona
        GROUP BY m_target.CVEGEO
    ) iso_influence ON m.CVEGEO = iso_influence.CVEGEO_manzana_objetivo;
    """

    print(f"Ejecutando cruce espacial para crear '{TABLE_OUTPUT_CRUCE}'...")
    con_cruce.execute(query_cruce_influencia)

    num_manzanas_resultado = con_cruce.execute(f"SELECT COUNT(*) FROM {TABLE_OUTPUT_CRUCE}").fetchone()[0]
    sum_influencia = con_cruce.execute(f"SELECT SUM(influencia_iso_farm_10min) FROM {TABLE_OUTPUT_CRUCE}").fetchone()
    sum_influencia_val = sum_influencia[0] if sum_influencia else 0

    print(f"Tabla '{TABLE_OUTPUT_CRUCE}' creada con {num_manzanas_resultado} manzanas.")
    print(f"Suma total de 'influencias de isócronas' en manzanas: {sum_influencia_val if sum_influencia_val else 0}")

    print("\nPrimeras filas de la tabla con el nuevo campo 'influencia_iso_farm_10min':")
    df_preview_final = con_cruce.execute(f"SELECT CVEGEO, NOM_MUN, POBTOT, num_farmacias_en_manzana, influencia_iso_farm_10min FROM {TABLE_OUTPUT_CRUCE} WHERE influencia_iso_farm_10min > 0 LIMIT 5").fetchdf()
    if not df_preview_final.empty:
        print(df_preview_final)
    else:
        df_preview_all = con_cruce.execute(f"SELECT CVEGEO, NOM_MUN, POBTOT, num_farmacias_en_manzana, influencia_iso_farm_10min FROM {TABLE_OUTPUT_CRUCE} LIMIT 5").fetchdf()
        print(df_preview_all)
        print(" (Ninguna de las primeras manzanas en la vista previa tenía influencia de isócronas).")

except ValueError as ve:
    print(f"Error de Valor: {ve}")
except duckdb.Error as de:
    print(f"Error de DuckDB: {de}")
    import traceback; traceback.print_exc()
except Exception as e:
    print(f"Error general en el cruce espacial de isócronas (Celda 14): {e}")
    import traceback; traceback.print_exc()
finally:
    if con_cruce:
        try:
            con_cruce.close()
            print("\nConexión a DuckDB cerrada al finalizar Celda 14.")
        except Exception as e_close:
            print(f"Error al cerrar la conexión en Celda 14: {e_close}")

print("\n--- Fin del Script de Cruce de Influencia de Isócronas ---")

In [ ]:
# Celda 15: Exportar Resultado Final a GeoPackage y Descargar
# --------------------------------------------------------------------------------------------

print("--- Iniciando Exportación a GeoPackage del Resultado Final (Corregido KeyError) ---")

DB_FILE_PATH_EXPORT_FINAL = "./inegi_analisis_01.duckdb"
TABLE_TO_EXPORT_FINAL = "manzanas_con_influencia_isocronas_final_ags"

GPKG_OUTPUT_DIR_FINAL = os.path.join(DIR_BASE_ISOCRONAS_MZA, "geopackages_final_output") # DIR_BASE_ISOCRONAS_MZA de Celda 11
os.makedirs(GPKG_OUTPUT_DIR_FINAL, exist_ok=True)

GPKG_FILENAME_FINAL = f"manzanas_ags_final_con_influencia_iso_farm.gpkg"
GPKG_FILE_PATH_FINAL = os.path.join(GPKG_OUTPUT_DIR_FINAL, GPKG_FILENAME_FINAL)
LAYER_NAME_FINAL_GPKG = "manzanas_con_influencia_isocronas"
CRS_EXPORT_FINAL = "EPSG:4326"

con_export_final = None

if os.path.exists(DB_FILE_PATH_EXPORT_FINAL):
    try:
        con_export_final = duckdb.connect(database=DB_FILE_PATH_EXPORT_FINAL, read_only=True)
        print("Instalando (si es necesario) y cargando la extensión 'spatial' para exportación final...")
        con_export_final.execute("INSTALL spatial;")
        con_export_final.execute("LOAD spatial;")
        print(f"Conexión a '{DB_FILE_PATH_EXPORT_FINAL}' establecida (read_only) y extensión 'spatial' lista.")

        table_exists_check_final = con_export_final.execute(f"SELECT 1 FROM information_schema.tables WHERE table_name = '{TABLE_TO_EXPORT_FINAL}'").fetchone()
        if not table_exists_check_final:
            print(f"¡ERROR! La tabla '{TABLE_TO_EXPORT_FINAL}' no existe. No se puede exportar.")
        else:
            print(f"Leyendo datos de '{TABLE_TO_EXPORT_FINAL}'...")
            df_to_export_final_duck = con_export_final.execute(f"SELECT *, ST_AsText(geometry) AS geom_wkt FROM {TABLE_TO_EXPORT_FINAL} WHERE geometry IS NOT NULL;").fetch_df()

            if df_to_export_final_duck.empty:
                print(f"La tabla '{TABLE_TO_EXPORT_FINAL}' está vacía o sin geometrías válidas. No se generará GeoPackage.")
            else:
                print(f"  Datos leídos. {len(df_to_export_final_duck)} filas. Procesando geometrías WKT...")

                def parse_wkt_for_final_export(wkt_string):
                    if wkt_string is None or not isinstance(wkt_string, str) or wkt_string.strip() == "": return None
                    try: return wkt.loads(wkt_string)
                    except: return None

                df_to_export_final_duck['geometry_parsed'] = df_to_export_final_duck['geom_wkt'].apply(parse_wkt_for_final_export)

                cols_to_drop_final = ['geom_wkt']
                if 'geometry' in df_to_export_final_duck.columns:
                    cols_to_drop_final.append('geometry')

                gdf_to_export_final = gpd.GeoDataFrame(
                    df_to_export_final_duck.drop(columns=cols_to_drop_final, errors='ignore'),
                    geometry='geometry_parsed', # La geometría activa es 'geometry_parsed'
                    crs=CRS_EXPORT_FINAL
                )

                # ***** CAMBIO AQUÍ: Renombrar ANTES de dropna, o usar 'geometry_parsed' en dropna *****
                gdf_to_export_final.rename_geometry('geometry', inplace=True) # Renombrar a 'geometry'
                gdf_to_export_final.dropna(subset=['geometry'], inplace=True) # AHORA sí existe la columna 'geometry'

                if gdf_to_export_final.empty:
                    print("GeoDataFrame final para exportar está vacío después de procesar geometrías WKT o eliminar NAs.")
                else:
                    print(f"GeoDataFrame final listo con {len(gdf_to_export_final)} filas. Guardando en '{GPKG_FILE_PATH_FINAL}'...")
                    gdf_to_export_final.to_file(GPKG_FILE_PATH_FINAL, layer=LAYER_NAME_FINAL_GPKG, driver="GPKG")
                    print("¡GeoPackage final (con influencia de isócronas) guardado exitosamente!")

                    from google.colab import files
                    print(f"\nSolicitando descarga de: {GPKG_FILE_PATH_FINAL}")
                    files.download(GPKG_FILE_PATH_FINAL)
    except Exception as e_export_final:
        print(f"Error durante la exportación final a GeoPackage: {e_export_final}")
        import traceback; traceback.print_exc()
    finally:
        if con_export_final:
            try: con_export_final.close(); print("Conexión de exportación final cerrada.")
            except: pass
else:
    print(f"La base de datos '{DB_FILE_PATH_EXPORT_FINAL}' no fue encontrada para la exportación final.")

print("\n--- Fin del Script de Exportación del GeoPackage Final (Corregido KeyError) ---")